In [164]:
import pandas as pd
from datetime import timedelta
from matplotlib import pyplot as plt
import numpy as np
from fastapi import FastAPI
import requests
import os

Start by making DF for average temperature per day

In [165]:
df = pd.read_csv('../data/smhi_temp_data_save.csv', usecols=[0, 1, 2, 3], header=6, sep=';')

In [166]:
df

,Datum,Tid (UTC),Lufttemperatur,Kvalitet
0,1944-01-01,07:00:00,1.6,G
1,1944-01-01,13:00:00,3.8,G
2,1944-01-01,18:00:00,4.2,G
3,1944-01-02,07:00:00,0.6,G
4,1944-01-02,13:00:00,3.8,G
...,...,...,...,...
435387,2006-12-05,09:00:00,9.4,G
435388,2006-12-05,10:00:00,10.1,Y
435389,2006-12-05,11:00:00,10.8,Y
435390,2006-12-05,12:00:00,10.9,G


In [167]:
df['Snittemperatur'] = df.groupby('Datum')['Lufttemperatur'].transform('mean').round(2)

In [168]:
df = df.groupby('Datum').max().drop('Lufttemperatur', axis=1).reset_index()

In [169]:
df['Year'] = df['Datum'].str.split('-').str[0]

In [170]:
df['Year'] = pd.to_datetime(df['Year'], yearfirst=True, format='%Y')

In [171]:
df['Datum'] = pd.to_datetime(df['Datum'], yearfirst=True, format='ISO8601')

In [172]:
df = df.drop(columns='Tid (UTC)')

In [173]:
df['Year']

0       1944-01-01
1       1944-01-01
2       1944-01-01
3       1944-01-01
4       1944-01-01
           ...    
22980   2006-01-01
22981   2006-01-01
22982   2006-01-01
22983   2006-01-01
22984   2006-01-01
Name: Year, Length: 22985, dtype: datetime64[ns]

In [174]:
df.to_csv('../data/Olof_Save_GBG_avg_daily_temp_data.csv', index=False, mode='w')

In [175]:
last_day_save = df['Datum'][-1:]

In [176]:
last_day_save = str(last_day_save.values[0])[:10]


In [177]:
df_gbg = pd.read_csv('../data/smhi_temp_data_gbg.csv', usecols=[0, 1, 2, 3], header=11, sep=';')

In [178]:
df_gbg

,Datum,Tid (UTC),Lufttemperatur,Kvalitet
0,1961-01-01,06:00:00,0.8,G
1,1961-01-01,12:00:00,1.0,G
2,1961-01-01,18:00:00,1.4,G
3,1961-01-02,06:00:00,1.8,G
4,1961-01-02,12:00:00,2.2,G
...,...,...,...,...
308145,2023-07-01,02:00:00,16.1,G
308146,2023-07-01,03:00:00,16.1,G
308147,2023-07-01,04:00:00,16.2,G
308148,2023-07-01,05:00:00,16.3,G


In [179]:
df_gbg['Snittemperatur'] = df_gbg.groupby('Datum')['Lufttemperatur'].transform('mean').round(2)

In [180]:
df_gbg = df_gbg.groupby('Datum').max().drop('Lufttemperatur', axis=1).reset_index()

In [181]:
df_gbg['Year'] = df_gbg['Datum'].str.split('-').str[0]

In [182]:
df_gbg['Year'] = pd.to_datetime(df_gbg['Year'], yearfirst=True, format='%Y')

In [183]:
df_gbg['Datum'] = pd.to_datetime(df_gbg['Datum'], yearfirst=True, format='ISO8601')

In [184]:
df_gbg = df_gbg.drop(columns='Tid (UTC)')

In [185]:
df_gbg = df_gbg[df_gbg['Datum'] > last_day_save]

In [186]:
df_gbg

,Datum,Kvalitet,Snittemperatur,Year
13892,2006-12-06,Y,9.29,2006-01-01
13893,2006-12-07,Y,7.70,2006-01-01
13894,2006-12-08,Y,8.86,2006-01-01
13895,2006-12-09,Y,8.58,2006-01-01
13896,2006-12-10,Y,7.52,2006-01-01
...,...,...,...,...
19909,2023-06-27,G,18.18,2023-01-01
19910,2023-06-28,G,19.72,2023-01-01
19911,2023-06-29,G,19.22,2023-01-01
19912,2023-06-30,G,17.40,2023-01-01


In [187]:
last_day_gbg= df_gbg['Datum'][-1:]


In [188]:
last_day_gbg = str(last_day_gbg.values[0])[:10]

In [189]:
df_gbg.to_csv('../data/Olof_Save_GBG_avg_daily_temp_data.csv', index=False, mode='a', header=False)

Compile historical data with API data

In [190]:
df_last_months = pd.read_csv('../data/smhi_gbg_last_few_months.csv', usecols=[0, 1, 2, 3], header=11, sep=';')

In [191]:
df_last_months

,Datum,Tid (UTC),Lufttemperatur,Kvalitet
0,2023-06-08,01:00:00,12.6,G
1,2023-06-08,02:00:00,12.2,G
2,2023-06-08,03:00:00,12.5,G
3,2023-06-08,04:00:00,12.4,G
4,2023-06-08,05:00:00,14.5,G
...,...,...,...,...
3125,2023-10-16,06:00:00,2.2,G
3126,2023-10-16,07:00:00,2.9,G
3127,2023-10-16,08:00:00,5.3,G
3128,2023-10-16,09:00:00,7.9,G


In [192]:
df_last_months['Snittemperatur'] = df_last_months.groupby('Datum')['Lufttemperatur'].transform('mean').round(2)

In [193]:
df_last_months = df_last_months.groupby('Datum').max().drop('Lufttemperatur', axis = 1).reset_index()

In [194]:
df_last_months['Year'] = df_last_months['Datum'].str.split('-').str[0]

In [195]:
df_last_months['Year'] = pd.to_datetime(df_last_months['Year'], yearfirst=True, format='%Y')

In [196]:
df_last_months['Datum'] = pd.to_datetime(df_last_months['Datum'], yearfirst=True, format='ISO8601')

In [197]:
df_last_months = df_last_months.drop(columns='Tid (UTC)')

In [198]:
df_last_months

,Datum,Kvalitet,Snittemperatur,Year
0,2023-06-08,G,17.67,2023-01-01
1,2023-06-09,G,19.30,2023-01-01
2,2023-06-10,G,18.98,2023-01-01
3,2023-06-11,G,18.52,2023-01-01
4,2023-06-12,G,19.63,2023-01-01
...,...,...,...,...
126,2023-10-12,G,12.08,2023-01-01
127,2023-10-13,G,11.89,2023-01-01
128,2023-10-14,G,11.54,2023-01-01
129,2023-10-15,G,10.09,2023-01-01


In [199]:
df_last_months.dtypes

Datum             datetime64[ns]
Kvalitet                  object
Snittemperatur           float64
Year              datetime64[ns]
dtype: object

In [200]:
last_day_gbg

'2023-07-01'

In [201]:
df_last_months = df_last_months[df_last_months['Datum'] > last_day_gbg]

In [202]:
df_last_months

,Datum,Kvalitet,Snittemperatur,Year
24,2023-07-02,G,15.98,2023-01-01
25,2023-07-03,G,14.69,2023-01-01
26,2023-07-04,G,14.72,2023-01-01
27,2023-07-05,G,15.52,2023-01-01
28,2023-07-06,G,15.01,2023-01-01
...,...,...,...,...
126,2023-10-12,G,12.08,2023-01-01
127,2023-10-13,G,11.89,2023-01-01
128,2023-10-14,G,11.54,2023-01-01
129,2023-10-15,G,10.09,2023-01-01


In [203]:
df_last_months.to_csv('../data/Olof_Save_GBG_avg_daily_temp_data.csv', index=False, mode='a', header=False)